In [0]:
dbutils.widgets.text(name="env",defaultValue="",label=" Enter the environment in lower case")
env = dbutils.widgets.get("env")


def create_bronze_schema(env):
    spark.sql(f"""USE CATALOG hive_metastore""")
    spark.sql(f"""
    CREATE SCHEMA IF NOT EXISTS dev_bronze""")


def create_fundamental_table(env,path):
    spark.sql(f"""CREATE TABLE IF NOT EXISTS hive_metastore.{env}_bronze.fundamentals (
    symbol STRING,
    shortName STRING,
    sector STRING,
    industry STRING,
    country STRING,
    currency STRING,
    fullTimeEmployees STRING,       -- was INT
    marketCap STRING,               -- was BIGINT
    enterpriseValue STRING,         -- was BIGINT
    totalRevenue STRING,            -- was BIGINT
    netIncomeToCommon STRING,       -- was BIGINT
    profitMargins DOUBLE,
    revenueGrowth DOUBLE,
    ebitda STRING,                  -- was BIGINT
    enterpriseToRevenue DOUBLE,
    enterpriseToEbitda DOUBLE,
    bookValue DOUBLE,
    priceToBook DOUBLE,
    trailingPE DOUBLE,
    forwardPE DOUBLE,
    trailingEps DOUBLE,
    forwardEps DOUBLE,
    returnOnAssets DOUBLE,
    returnOnEquity DOUBLE,
    earningsQuarterlyGrowth DOUBLE,
    ipoExpectedDate STRING, 
    extract_time TIMESTAMP

    )
    USING DELTA
    PARTITIONED BY (extract_time)
    LOCATION '{path}/{env}/bronze/fundamentals';""")

def create_prices_table(env, path):
    spark.sql(f"""
    CREATE TABLE IF NOT EXISTS hive_metastore.{env}_bronze.prices (
        symbol STRING,
        current_price DOUBLE,
        open DOUBLE,
        day_high DOUBLE,
        day_low DOUBLE,
        previous_close DOUBLE,
        volume DOUBLE,
        market_cap DOUBLE,
        extract_time TIMESTAMP
    )
    USING DELTA
    PARTITIONED BY (extract_time)
    LOCATION '{path}/{env}/bronze/prices';
    """)

bronze_path = 'abfss://stock-project-container@bgardzinski1stock.dfs.core.windows.net'
create_bronze_schema(env)
create_fundamental_table(env,bronze_path)
create_prices_table(env,bronze_path)


In [0]:
%sql DROP TABLE IF EXISTS hive_metastore.dev_bronze.stock_prices;
